In [1]:
from slicer import *

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Experimental workflow for the paper

Next is the experiments performed in the HAIS 2021 work "Slicer: Feature Learning for Class Separability with Least-Squares Support Vector Machine Loss and COVID-19 Chest X-ray Case Study".

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

cls = {"dt": DecisionTreeClassifier, "knn": KNeighborsClassifier, "svm": SVC, "gp": GaussianProcessClassifier}
pars = {"dt": {"random_state": 123456, "max_depth": 10}, "knn": {}, "svm": {}, "gp": {}}
metrics = {
    "acc": accuracy_score,
    "prec": precision_score,
    "rec": recall_score,
    "f1": f1_score
}

def flatten(x):
    return np.reshape(x, (x.shape[0], np.prod(x.shape[1:])))

def train_test(cl, x_train, y_train, x_test, y_test, metrics={}, *args, **kwargs):
    """
    Classifier training and test function (applicable to any sklearn classifier)
    """
    x_train = flatten(x_train)
    x_test = flatten(x_test)
    model = cl(*args, **kwargs)
    model.fit(x_train, y_train)
    p_test = model.predict(x_test)
    return {
        mname: met(y_test, p_test)
        for mname, met in metrics.items()
    }

In [12]:
cl = GaussianProcessClassifier().fit(flatten(x_train), y_train)

In [15]:
cl.predict(flatten(x_val))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [16]:
y_val

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

Repeated cross validation (5 times 5-fold) loop:

In [4]:
instance_shape = (512, 512, 1)
for cv in [1, 2, 4, 5]:
    #cv = 3
    results_file = f"slicer_results_cv{cv}.csv"
    with open(results_file, "w") as outfile:
        outfile.write(f"partition, classifier, preprocessing, {', '.join(met for met in metrics.keys())}\n")
        for p in range(5):
            datagen = DataLoader(cv, p, instance_shape)

            res_dict = train_predict(datagen)
            x_train, y_train, x_t_ae, x_t_enc = res_dict["train"]
            x_val, y_val, x_v_ae, x_v_enc = res_dict["test"]

            for clname, cl in cls.items():
                results = train_test(cl, x_train, y_train, x_val, y_val, metrics=metrics, **pars[clname])
                results_str = ", ".join(str(met) for met in results.values())
                outfile.write(f"{p}, {clname}, original, {results_str}\n")
                results = train_test(cl, x_t_enc, y_train, x_v_enc, y_val, metrics=metrics, **pars[clname])
                results_str = ", ".join(str(met) for met in results.values())
                outfile.write(f"{p}, {clname}, slicer, {results_str}\n")
                results = train_test(cl, x_t_ae, y_train, x_v_ae, y_val, metrics=metrics, **pars[clname])
                results_str = ", ".join(str(met) for met in results.values())
                outfile.write(f"{p}, {clname}, autoencoder, {results_str}\n")
            outfile.flush()

Found 614 images belonging to 2 classes.
Found 614 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Found 170 images belonging to 2 classes.
Conv encoding: (8, 8, 32), internal encoding: 128
Conv encoding: (8, 8, 32), internal encoding: 128
Epoch 0 / time taken: 38.21264696121216, train loss: 0.6539493203163147, val loss: 0.7047253847122192
Epoch 1 / time taken: 34.0575590133667, train loss: 0.6354503035545349, val loss: 0.718813955783844
Epoch 2 / time taken: 34.31102228164673, train loss: 0.6297346949577332, val loss: 0.6598015427589417
Epoch 3 / time taken: 34.10560464859009, train loss: 0.6359542608261108, val loss: 0.643825888633728
Epoch 4 / time taken: 34.144550800323486, train loss: 0.6169059872627258, val loss: 0.638533353805542
Epoch 5 / time taken: 34.339492321014404, train loss: 0.6294590830802917, val loss: 0.6318482160568237
Epoch 6 / time taken: 34.1010308265686, train loss: 0.6000087857246399, val loss: 0.6232494115829468
Epoch 7 / time taken: 33.9

/home/david/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/david/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 614 images belonging to 2 classes.
Found 614 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Found 170 images belonging to 2 classes.
Conv encoding: (8, 8, 32), internal encoding: 128
Conv encoding: (8, 8, 32), internal encoding: 128
Epoch 0 / time taken: 37.01208472251892, train loss: 0.6451863646507263, val loss: 0.7024486064910889
Epoch 1 / time taken: 33.18065309524536, train loss: 0.625979483127594, val loss: 0.6751760840415955
Epoch 2 / time taken: 33.271923542022705, train loss: 0.620231568813324, val loss: 0.6487526297569275
Epoch 3 / time taken: 33.38495469093323, train loss: 0.6152645349502563, val loss: 0.6311081647872925
Epoch 4 / time taken: 33.2803521156311, train loss: 0.620944082736969, val loss: 0.6270425915718079
Epoch 5 / time taken: 33.80154037475586, train loss: 0.6278173327445984, val loss: 0.6239431500434875
Epoch 6 / time taken: 33.89452624320984, train loss: 0.6231815218925476, val loss: 0.6272839903831482
Epoch 7 / time taken: 33.5

/home/david/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 614 images belonging to 2 classes.
Found 614 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Found 170 images belonging to 2 classes.
Conv encoding: (8, 8, 32), internal encoding: 128
Conv encoding: (8, 8, 32), internal encoding: 128
Epoch 0 / time taken: 37.182576417922974, train loss: 0.6459250450134277, val loss: 0.6954701542854309
Epoch 1 / time taken: 33.36161279678345, train loss: 0.6375979781150818, val loss: 0.7404325604438782
Epoch 2 / time taken: 33.4776074886322, train loss: 0.6366621851921082, val loss: 0.738326370716095
Epoch 3 / time taken: 33.4371063709259, train loss: 0.6291296482086182, val loss: 0.6758916974067688
Epoch 4 / time taken: 33.672287940979004, train loss: 0.6190152764320374, val loss: 0.6462587118148804
Epoch 5 / time taken: 33.451311111450195, train loss: 0.6200349926948547, val loss: 0.6329771876335144
Epoch 6 / time taken: 33.43650817871094, train loss: 0.6271608471870422, val loss: 0.6255834102630615
Epoch 7 / time taken: 3

...

/home/david/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 614 images belonging to 2 classes.
Found 614 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Found 170 images belonging to 2 classes.
Conv encoding: (8, 8, 32), internal encoding: 128
Conv encoding: (8, 8, 32), internal encoding: 128
Epoch 0 / time taken: 36.39581871032715, train loss: 0.6411614418029785, val loss: 0.6935039162635803
Epoch 1 / time taken: 32.646260023117065, train loss: 0.6286155581474304, val loss: 0.6713492274284363
Epoch 2 / time taken: 32.80931353569031, train loss: 0.6316154599189758, val loss: 0.6446332335472107
Epoch 3 / time taken: 32.76769518852234, train loss: 0.6177971959114075, val loss: 0.6344873905181885
Epoch 4 / time taken: 32.497995138168335, train loss: 0.6218823790550232, val loss: 0.6299164891242981
Epoch 5 / time taken: 32.562451124191284, train loss: 0.6398143768310547, val loss: 0.6245273947715759
Epoch 6 / time taken: 32.71223068237305, train loss: 0.6187408566474915, val loss: 0.6219689846038818
Epoch 7 / time taken

/home/david/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 612 images belonging to 2 classes.
Found 612 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Found 172 images belonging to 2 classes.
Conv encoding: (8, 8, 32), internal encoding: 128
Conv encoding: (8, 8, 32), internal encoding: 128
Epoch 0 / time taken: 36.43262839317322, train loss: 0.6378332376480103, val loss: 0.7096501588821411
Epoch 1 / time taken: 32.703588008880615, train loss: 0.6393255591392517, val loss: 0.7010583281517029
Epoch 2 / time taken: 32.9136803150177, train loss: 0.6204559206962585, val loss: 0.6573649644851685
Epoch 3 / time taken: 33.14526605606079, train loss: 0.6255242228507996, val loss: 0.6468347907066345
Epoch 4 / time taken: 32.852869749069214, train loss: 0.6203504204750061, val loss: 0.637831449508667
Epoch 5 / time taken: 33.22040057182312, train loss: 0.6105117201805115, val loss: 0.6331347227096558
Epoch 6 / time taken: 33.51396059989929, train loss: 0.6210750341415405, val loss: 0.6259806156158447
Epoch 7 / time taken: 3

/home/david/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
